<center><b>Spatial Data Vizualization Assignment 2</b></center>


<b>Part 1: Process the csv to be used in blender</b>

In [2]:
import pandas as pd
from datetime import datetime

# Load in and clean up csv
bird_data = pd.read_csv('bird_data.csv')
remove_columns = ['visible', 'location-long', 'location-lat', 'comments', 'sensor-type', 
                  'individual-taxon-canonical-name', 'tag-local-identifier', 'study-name', 'utm-zone']
bird_data.drop(columns=remove_columns, inplace=True)
bird_data.columns = ['event-id', 'timestamp', 'bird-id', 'x', 'y']

# Convert timestamp to unix
def convert_to_unix(timestamp):
   dt = datetime.strptime(timestamp, '%Y-%m-%d %H:%M:%S.%f')
   unix_time = int((dt - datetime(1970, 1, 1)).total_seconds())
   return unix_time
bird_data['timestamp'] = bird_data['timestamp'].apply(convert_to_unix)

# Change timestamp to time elapsed
def time_elasped(group):
   start = group.index[0]
   group['timestamp'] = group['timestamp'].astype(int)
   group['timestamp'] -= group.at[start,'timestamp']
   return group
bird_data = bird_data.groupby('bird-id', group_keys=True).apply(time_elasped).reset_index(drop=True)

# Define boundary box for "projection" (UTM 31N)
xmin = bird_data['x'].min()
xmax = bird_data['x'].max()
ymin = bird_data['y'].min()
ymax = bird_data['y'].max()

# Reassign coordinates (0-max) for blender
bird_data['x'] = bird_data['x'] - xmin
bird_data['y'] = bird_data['y'] - ymin

# View dataframe
print(bird_data)
# Save to csv
#bird_data.to_csv('bird4blender.csv')

         event-id  timestamp           bird-id              x              y
0      3519810783          0   anosmic-6207703  299064.178993  112159.839352
1      3519810784        300   anosmic-6207703  296482.930005  111237.429726
2      3519810785        600   anosmic-6207703  293904.469986  110196.959584
3      3519810786        900   anosmic-6207703  291311.259007  109225.839592
4      3519810787       1200   anosmic-6207703  289672.670965  108683.869886
...           ...        ...               ...            ...            ...
54555  3519842262     669060  magnetic-6220627  302240.738009  113868.439310
54556  3519842263     669360  magnetic-6220627  302243.047007  114045.809807
54557  3519842264     669660  magnetic-6220627  302338.169021  113338.059878
54558  3519842265     669960  magnetic-6220627  302404.248989  112723.909422
54559  3519842266     670260  magnetic-6220627  302343.632966  112838.969430

[54560 rows x 5 columns]
Unique bird-ids: ['anosmic-6207703', 'anosmic-6212

<b>Part 2: Create the bird path space-time cube in blender</b>

Paste into blender text editor and run

In [ ]:
import bpy
import csv
import bmesh
from mathutils import Vector

# Load in processed bird data
bird_data = 'C:/Users/Austin/Documents/DATABANK/Masters/SpatDataViz/bird4blender.csv'

# Scale coordinates of data to blender size
original_x_range = (0, 396945.09997829294)
original_y_range = (0, 324946.84898422007)
original_timestamp_range = (0, 1136520)
target_range = (-10, 10)
def scale_coordinates(coord, original_range, target_range):
    min_orig, max_orig = original_range
    min_target, max_target = target_range
    return min_target + ((coord - min_orig) / (max_orig - min_orig)) * (max_target - min_target)

# Get coordinates 
vertex_groups = {}

with open(bird_data, newline='') as csvfile:
    csv_reader = csv.DictReader(csvfile)

    for row in csv_reader:
        bird_id = row['bird-id']
        
        x = float(row['x'])
        y = float(row['y'])
        timestamp = float(row['timestamp'])
        scaled_x = scale_coordinates(x, original_x_range, target_range)
        scaled_y = scale_coordinates(y, original_y_range, target_range)
        scaled_timestamp = scale_coordinates(timestamp, original_timestamp_range, target_range)
        
        # Seperate the points into bird_id vertex groups 
        if bird_id not in vertex_groups:
            vertex_groups[bird_id] = []
        vertex_groups[bird_id].append(Vector((scaled_x, scaled_y, scaled_timestamp)))

def get_color_from_bird_id(bird_id):
    bird_id_class = bird_id.split('-')[0].strip()
    if bird_id_class == 'control':
        return (1, 0, 0, 1), 'control'  # Red
    elif bird_id_class == 'magnetic':
        return (0, 1, 0, 1), 'magnetic'  # Green
    elif bird_id_class == 'anosmic':
        return (0, 0, 0, 0), 'anosmic'  # Black
    else:
        return (0.5, 0.5, 0.5, 1)

bpy.ops.object.mode_set(mode='OBJECT')

# Iterate over every bird_id vertex group
for bird_id, group_vertices in vertex_groups.items():
    # Add curve object
    bpy.context.scene.cursor.location = group_vertices[0]
    bpy.ops.curve.primitive_bezier_curve_add(radius=1, location=(0, 0, 0))
    curve_object = bpy.context.object
    curve = curve_object.data
    # Iterate over all points to form curve to data
    for i, vertex in enumerate(group_vertices[1:]):
        curve.splines[0].bezier_points.add(count=1)
        curve.splines[0].bezier_points[i].co = vertex
    
    # Curve smoothness
    curve.resolution_u = 1
    # Curve radius
    curve.bevel_depth = 0.05
    
    # Get color for each category of birds
    color = get_color_from_bird_id(bird_id)
    
    # Use existing material if it exists
    material_name = f"Material_{color[1]}"
    existing_material = bpy.data.materials.get(material_name)
    if existing_material:
        material = existing_material
    # Otherwise create new material for that bird type
    else:
        # Emission shader node tree
        material = bpy.data.materials.new(name=material_name)
        material.use_nodes = True
        material.node_tree.nodes.clear()
        emission_node = material.node_tree.nodes.new(type='ShaderNodeEmission')
        emission_node.location = (0, 0)
        
        emission_node.inputs['Color'].default_value = color[0]
        material_output = material.node_tree.nodes.new(type='ShaderNodeOutputMaterial')
        material_output.location = (400, 0)
        material.node_tree.links.new(emission_node.outputs['Emission'], material_output.inputs['Surface'])
    
    # Apply material to curve
    curve.materials.append(material)
        

bpy.ops.object.mode_set(mode='EDIT')
bmesh.update_edit_mesh(mesh)

<b>Part 3: Animate Progression of Time</b>

Paste into blender text editor and run. Select a curve, go into edit mode, and play animation or scrub along the timeline.

In [ ]:
import bpy

def reveal_points(scene):
    # Only works once you select a curve
    if bpy.context.active_object and bpy.context.active_object.type == 'CURVE':
        
        curve = bpy.context.active_object.data
        frame = scene.frame_current

        # Calculate the number of points to show
        points_per_frame = 25
        points_to_show = (frame * points_per_frame)
        
        # Hide points out of range
        for i, point in enumerate(curve.splines[0].bezier_points):
            point.hide = i >= points_to_show

# Use handlers to detect a frame change and update the points reveal for each frame
def register_handlers():
    bpy.app.handlers.frame_change_pre.clear()

    bpy.app.handlers.frame_change_pre.append(reveal_points)

# Call the register handlers function
register_handlers()